In [2]:
import numpy as np
import math

# Bursty and Hierarchical Structure in Streams

Perhaps the simplest randomized model for generating a sequence of message arrival times is based on an exponential distirbution: messages are emitted in a probabilistic manner, so that the gap $x$ in time between messages $i$ and $i+1$ is distributed according to the "memoryless" exponential density function $f(x) = \alpha e^{-\alpha x}$ for a parameter $\alpha > 0$. (In other words, the probability that the gap exceeds $x$ is equal to $e^{-\alpha x}$.) The expected value of the gap i this model is $\alpha^{-1}$, and hence one can refer to $\alpha$ as the *rate* of message arrivals.  

Inteutively, a "bursty" model should extend this simple formulation by exhibiting periods of lower rate interlaved with periods of higher rate. A natural way to do this is to construct a model whith multiple *states*, where the rate depends on the current state. Let us star with a basic model that incorporates this idea, and then extend it to the models wthat will primarily be used in what follows.

**A two state model.** Arguably the most basic bursty model of this type would be constructed from a probabilistic automaton $A$ with two states $q_0$ and $q_1$, which we can think of as corresponding to "low" or "high". When $A$ is in state $q_0$, messages are emitted at slow rate, with gaps $x$ between consecutive messages distributed indepently accodring to a density function $f_0(x) = \alpha_0e^{-\alpha_0 x}$. When $A$ is in state $q_1$, messages are emitted ata a gaster rate, with gaps distributed independtly accordig to $f_1(x) = \alpha_1e^{-\alpha_1 x}$, where $\alpha_1>\alpha_0$. Finally, between messages, $A$ changes state with probability $p \in (0,1)$, remaining in its current state with probability $p = 1-p$ indepently of previous emitions and state changes.

Such a model could be used to generate a sequence a messages in the natural way. $A$ begins in state $q_0$. Before each message (including the first) is emitted, $A$ changes state with probability $p$. A message is then emitted, and the gap in time until the next message is determined by the distribution associatade with $A$'s current state.  

One can apply this generative model to find a likely state sequence given a set of messages. Suppose there is a given set of $n + 1$ messages, with specified arrival times; this determines a sequence of $n$ *inter-arrival* gaps $x=(x_1, ..., x_n)$. The development here will use the basic assumption that **all gaps** $x_i$ **are strictly possitive**. We can use the Bayes Procedure (as in [Nearest neighbor pattern classification](http://ieeexplore.ieee.org/document/1053964/)) to determine the conditional probability of a state sequence $q = (q_{i_1},...,q_{i_n})$; note that this must be done in terms of the underlying density functions, since the gaps are not drawn from discrete distributions. Each state sequence $q$ induces a density function $f_q$ over squences of gaps. which has the form $f_q(x_1,...,x_n) = \Pi_{t=1}^n f_{i_t}(x_t)$. If $b$ denotes the number of state transitions in the sequence $q$ - that is, the number of indices $i_t$ so that $q_{i_t} \neq q_{i_{t+1}}$ - then the (prior) probability of $q$ is equal to

$$(\prod_{i_t \neq i_{t+1}} p)(\prod_{i_t = i_{t+1}} 1-p) = p^b (1-p)^{n-b} = (\frac{p}{1-p})^b (1-p)^n$$

(In this calculation, let $i_0 = 0$, since $A$ starts in state $q_0$.) Now,

$$Pr[q\mid x] = \frac{Pr[q] f_q(x)}{\sum_{q'} Pr[q'] f_{q'}(x)} = \frac{1}{Z} (\frac{p}{1-p})^b (1-p)^n \prod_{t=1}^n f_{i_t}(x_t),$$

where $Z$ is the normalizing constant $\sum_{q'} Pr[q'] f_{q'}(x)$. Finding a state sequence $q$ maximizing this probability is equivalent to finding one that minimizes

$$-\ln Pr[q\mid x]$$

$$= b \ln \left(\frac{1-p}{p}\right)+\left(\sum_{t=1}^n -\ln f_{i_t}(x_t)\right) - \ln (1-p) + \ln Z.$$

Since the third and fourth terms are independent of the state sequence, this latter optimization problem is equivalent to finding a state sequence $q$ that minimizes the following *cost function*:

$$c(q \mid x) = b \left(\frac{1-p}{p}\right) + \left(\sum_{t=1}^n -\ln f_{i_t}(x_t)\right)$$

Finding a state sequence to minimize thisc cost function is a problem that can be motivated intuetivly on its own terms, without recourse to the underlying probabilistic model. The first of the two terms in the expression for $c(q \mid x)$ favors sequences with a small number of state transitions, while the second terms favor state sequences that conform well to the sequence $x$ of gap values. Thus, one expects the optimum to tracj the global structure of bitsts in the gap sequence, while holding to a single state thoriught local periods of non-uniformity. Varying the coefficient on $b$ controls the amount of "inertia" fixin the automaton in its current state. 

The next step is to extend this simple "high-low" model to one with a richer state set, using a cost model: this will lead to a method that alos extracts hierarchical structure from the pattern of bursts.

**An infinite-state model.** Consider a sequence of $n+1$ messages (code: `offsets`) that arrive over a period of time of length $T$ (code: `T`).

In [3]:
offsets = [1,2,3,4,5,6,7]  # n+1 message arrival times
gaps = np.diff(offsets)
T = np.sum(gaps)
n = np.size(gaps)

If the messages were spaced completely evenly over this time interval, then they would arrive with gaps of size $\hat{g} = T/n$ (code: `g_hat`).

In [4]:
g_hat = T / n

Bursts of greather and greater intensity would be associated with gaps of smaller and smaller than $\hat{g}$. This suggests focusing on an infinite-state automaton whose states correspond to gap sizes that may be arbitrarily small, so as to capture the full range of possible bursts. The development here will ise a cost model as in the two-state case, where the underlying goal is to find a state sequence of minimum cost.  

Thus, consider an automaton with a "base state" $q_0$ that has an exponential density function of $f_0$ with rate $\alpha_0 = \hat{g}^{-1} = n / T$ - consistent with completelely uniform message arrivals. For each $i>0$, there is a state $q_1$ with associated exponential density $f_i$ having rate $\alpha_i = \hat{g}^{-1} s^i$ (code: calculated by `alpha_function`), **where $s>1$ is a scaling parameter** (code: `s`).

In [5]:
s = 2
alpha_function = np.vectorize(lambda x: s ** x / g_hat)

($i$ will be referred to as the *index* of state $q_i$.) In other words, the infinite sequence of states $q_0, q_1, ...$ models the inter-arrival gaps that decrease geometrically from $\hat{g}$; there is ane xpected rate of message arrivals that intensifies for larger and larger values of $i$. Finally for every value of $i$ and $j$ **there is a cost** $\tau(i,j)$ (code: `tau`) **associated with a state transition from** $q_i$ to $q_j$**.** The framework allows considerable flexibility in formulating the cost function; for the work described here, $\tau(·,·)$ is defined so that the cost of moving from a lower-intensity burst to a higher-intensity one is a proportional to the number of intervening states, but ther eis no cost for the automaton to end a higher-intensity birst and drop down to a lower-intensity one. Specifically, **when** $j > i$**, moving from** $q_i$ to $q_j$** incurrs a cost of** $(j - i)\gamma \ln n$ where $\gamma > 0$ is a parameter (code: `gamma`); and when $j < i$ the cost is 0. 

In [6]:
gamma = 1
gamma_log_n = gamma * math.log(n)

def tau(i, j):
    if i >= j:
        return 0
    else:
        return (j - i) * gamma_log_n

This automaton, with its associated parameter $s$ and $\gamma$ will be denothed $A^{*}_{s,\gamma}$. Given a sequence of positive gaps $x=(x_1,x_2,...,x_n)$ between message arrivals, the goal - by analogy with the two-state model above- is to find a sequence $q = (q_{i_1},...,q_{i_n})$ that minimizes the cost function

$$c(q\mid x) = \left( \sum_{t=0}^{n-1} \tau(i_t,i_{t+1}) \right) + \left( \sum_{t=1}^{n} - \ln f_{i_t}(x_t)\right)$$

(Let $i_0 = 0$ in this expression so that $A^{*}_{s,\gamma}$, starts in the state $q_0$.) Since the set of possible $q$ is infinite, one cannot automatically assert that the minimum is even well-defined; but this will be stablished in Theorem 2.1 below. As before, minimizing the first erm is consistent with having a few state transitions - and transitions that span only a few distinct states- while minimizing the second term is consistent eiwth passing through states whose rartes rarely agree closely with the inter arrival gaps. Thus, the combined goals is to track the sequence of gaps as well as possible without changing state so much.

Observe that the scaling parameter $s$ controls the "resolution" with wich the discrete rate values of the staes are able to trach the real-valued gaps; the parameter $\gamma$ controls the ease with which the automaton can change state. In what follows, $\gamma$ will often be set to a default value of $1$, we can use $A^{*}_{s}$ to denote $A^{*}_{s,1}$

**Computing a minimum-cost state sequence.** Given a sequence of positive gaps $x=(x_1, ..., x_n)$ between message arrivals, consider te algoritmic problem of finding a state sequence $q = (q_{i_1},...,q_{i_n})$ in $A^{*}_{s,\gamma}$ in $A^{*}_{s,\gamma}$ that minimizes the cost $c(q \mid x)$; such a sequence will be called *optimal*. To establish that the minimum is well-defined, and to provide a means of computing it, it is useful to first define a natural finite restriction of the automation: for a natural number $k$, one simply deletes all states but $q_0, q_1, ..., q_{k-1}$ from $A^{*}_{s,\gamma}$, and denotes the resulting $k$-state automation by $A^{k}_{s,\gamma}$. Note that the two state autometon $A^{2}_{s,\gamma}$ is essentially equivalent (by an amortization argument) to the probabilistic two-state model described earlier.

It is not hard to show that computing an optimal state sequence in $A^{*}_{s,\gamma}$ is equivalent to doing so in one of its finite restrictions.

**Theorem 2.1**. Let $\delta(x) = min_{i=1}^n x_i$ (code: `delta_x`) and (code: `k`)

$$k = \lceil  i + \log_s T + \log_s \delta(x)^{-1}\rceil$$

(Note that $\delta(x) > 0$ since all gaps are positive.) If $q^*$ is an optimal state sequence in $A^{k}_{s,\gamma}$, then it is also an optimal state sequence in $A^{*}_{s,\gamma}$.

In [7]:
delta_x = np.amin(gaps)
k = int(math.ceil(float(1 + math.log(T, s) + math.log(1 / delta_x, s))))

In the view of the theorem, it is enough to give an algorithm that computes an optimal state sequence in an automaation of the form $A^{k}_{s,\gamma}$. This can be done by adapting the standard forward dynamic programming algorithm used for hidden Markov models to the model and cost function defined here: One defines $C_j(t)$ to be the minimum cost of a state sequence for the input $x_1, x_2, ..., x_t$ that must end with state $q_j$, and then iteratively builds up the values of $C_j(t)$ in order of increasing $t$ using the recurrence relation $C_j(t) = - \ln f_j(x_t) + min_l(C_l(t-1) + \tau(l, j))$ with initial conditions $C_o(0) = 0$ and $C_j(0) = \infty$ for $j > 0$. In all the experiments here, an optimal state sequence in $A^{k}_{s,\gamma}$ can be found by restricting to a number of states $k$ that is a very small constant, always at most $25$.  

In [8]:
C = np.repeat(float("inf"), k)
C[0] = 0

Note that although the final computation of an optimal state sequence is carried out by recoursing to a finite-state model, working with the infinite model has the advantage that a number of states $k$ is not fixed *a priori*; rather, it emerges in the course of the computation, and in this way the automaton $A^{k}_{s,\gamma}$ essentially "conforms" to the particular input stance.